In [21]:
import numpy as np
import tensorflow as tf

In [254]:
tf.reset_default_graph()

x = tf.Variable(np.array([
    0, 0, 7, 0, 0, 0, 3, 0, 2,
    2, 0, 0, 0, 0, 5, 0, 1, 0,
    0, 0, 0, 8, 0, 1, 4, 0, 0,
    0, 1, 0, 0, 9, 6, 0, 0, 8,
    7, 6, 0, 0, 0, 0, 0, 4, 9,
    0, 0, 0, 0, 0, 0 ,0, 0, 0,
    0, 0, 0, 1, 0, 3, 0, 0, 0,
    8, 0, 1, 0, 6, 0, 0, 0, 0,
    0, 0, 0 ,7, 0, 0, 0 ,6, 3
]).reshape(9,9))

dom_zero = tf.equal(x, 0)
n = tf.reduce_sum(tf.cast(dom_zero, tf.int32))
dom0 = tf.broadcast_to(dom_zero, (9,9,9))
domH = tf.stack([
    tf.tile(
        tf.reshape(
            tf.logical_not(tf.reduce_any(tf.equal(x, v), 1)),
        (9, 1)),
    (1, 9)) for v in range(1,10)
])
domV = tf.stack([
    tf.reshape(
        tf.tile(
            tf.logical_not(tf.reduce_any(tf.equal(x, v), 0)),
        (9,)),
    (9,9)) for v in range(1,10)
])
domB = tf.reshape(tf.transpose(
    tf.reshape(tf.tile(
        tf.equal(
            tf.transpose(tf.nn.max_pool(
                tf.reshape(
                    tf.stack([
                        tf.cast(tf.equal(x, v), tf.int32)
                            for v in range(1,10)
                    ], axis=2),
                (1,9,9,9)),
                (1,3,3,1),
                (1,3,3,1),
                'VALID',
            ), (3,1,2,0)),
        0), # shape: 9,3,3,1
    (1,1,1,9)), (9,3,3,3,3)),
(0,1,3,2,4)), (9,9,9))
dom = tf.cast(tf.logical_and(
    dom0, tf.logical_and(
        domB, tf.logical_and(
            domH, domV
        )
    )
), tf.int32)
ct0 = tf.where(tf.equal(tf.reduce_sum(dom, 0), 1))
g0 = tf.argmax(tf.gather_nd(tf.transpose(dom, (1,2,0)), ct0), axis=1) + 1

ctV = tf.where(tf.equal(tf.reduce_sum(dom, 1), 1))
gV = tf.argmax(tf.gather_nd(tf.transpose(dom, (0,2,1)), ctV), axis=1)
locV = tf.stack([gV, ctV[:, 1]], axis=1)
valV = ctV[:, 0] + 1

ctH = tf.where(tf.equal(tf.reduce_sum(dom, 2), 1))
gH = tf.argmax(tf.gather_nd(dom, ctH), axis=1)
locH = tf.stack([ctH[:, 1], gH], axis=1)
valH = ctH[:, 0] + 1

idxUpdates = tf.concat([ct0, locV, locH], axis=0)
valUpdates = tf.concat([g0, valV, valH], axis=0)

update_op = tf.scatter_nd_update(x, idxUpdates, valUpdates)

init = tf.global_variables_initializer()


In [255]:
with tf.Session() as sess:
    sess.run(init)
    for _ in range(sess.run(n)):
        idx, val, _ = sess.run([idxUpdates, valUpdates, update_op])
        for i, v in zip(idx, val):
            print("[{}, {}] ← {}".format(i[0], i[1], v))
    print(sess.run(x))
        

[0, 4] ← 4
[0, 0] ← 1
[5, 8] ← 1
[5, 5] ← 7
[0, 0] ← 1
[8, 6] ← 1
[2, 4] ← 2
[7, 1] ← 3
[0, 5] ← 9
[4, 4] ← 1
[6, 1] ← 7
[1, 4] ← 7
[4, 4] ← 1
[0, 3] ← 6
[5, 6] ← 6
[1, 3] ← 3
[1, 8] ← 6
[5, 4] ← 3
[7, 3] ← 9
[3, 7] ← 3
[4, 2] ← 3
[2, 0] ← 3
[2, 0] ← 3
[3, 6] ← 7
[4, 5] ← 8
[6, 0] ← 6
[2, 2] ← 6
[8, 4] ← 8
[6, 4] ← 5
[6, 4] ← 5
[6, 8] ← 4
[7, 5] ← 4
[8, 5] ← 2
[8, 5] ← 2
[5, 1] ← 2
[5, 7] ← 5
[6, 2] ← 2
[3, 3] ← 2
[5, 2] ← 8
[0, 7] ← 8
[4, 3] ← 5
[4, 6] ← 2
[5, 3] ← 4
[7, 7] ← 2
[5, 3] ← 4
[4, 3] ← 5
[7, 6] ← 5
[4, 6] ← 2
[0, 1] ← 5
[4, 3] ← 5
[5, 0] ← 9
[1, 6] ← 9
[2, 1] ← 9
[6, 7] ← 9
[7, 8] ← 7
[2, 8] ← 5
[2, 7] ← 7
[1, 1] ← 8
[6, 6] ← 8
[2, 8] ← 5
[7, 8] ← 7
[1, 1] ← 8
[6, 6] ← 8
[1, 2] ← 4
[8, 1] ← 4
[8, 1] ← 4
[8, 2] ← 9
[1, 2] ← 4
[8, 2] ← 9
[3, 2] ← 5
[8, 0] ← 5
[3, 0] ← 4
[3, 2] ← 5
[3, 0] ← 4
[8, 0] ← 5
[[1 5 7 6 4 9 3 8 2]
 [2 8 4 3 7 5 9 1 6]
 [3 9 6 8 2 1 4 7 5]
 [4 1 5 2 9 6 7 3 8]
 [7 6 3 5 1 8 2 4 9]
 [9 2 8 4 3 7 6 5 1]
 [6 7 2 1 5 3 8 9 4]
 [8 3 1 9 6 4 5 2 7]
 [5 4 9